In [73]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas as hvplot
import matplotlib.pyplot as plt 

import numpy as np  
import requests 
import json 

from citipy import citipy
from api_keys import yelp_api_key

### Generate the NYC Borough Grocery Stores List by Using Yelp API

In [74]:
api_key = yelp_api_key
base_url = "https://api.yelp.com/v3/businesses/search"
cities = ["Manhattan", "Queens", "Brooklyn", "Bronx", "Staten Island"]
results_per_page = 20  # Number of results per API request
all_eats_data = []
for city in cities:
    offset = 0  # Start with 0 offset
    while True:
        params = {
            "term": "restaurants",
            "location": city,
            "offset": offset,  # Offset for pagination
            "limit": results_per_page  # Number of results per page
        }
        headers = {
            "Authorization": f"Bearer {api_key}"
        }
        response = requests.get(base_url, params=params, headers=headers)
        eats_data = response.json()
        # Process the data as needed
        businesses = eats_data.get("businesses", [])
        if not businesses:
            break  # No more businesses, exit the loop
        for business in businesses:
            data = {
                "City": city,
                "Name": business["name"],
                "Address": business["location"]["address1"]
            }
            all_eats_data.append(data)
        offset += results_per_page  # Increment offset for the next page
# Create a DataFrame from the collected data
eats_data_df = pd.DataFrame(all_eats_data)
# Save the DataFrame to a CSV file
eats_data_df.to_csv("nyc_eats.csv", index=False)

In [75]:
# Read the groceries.csv file and create a groceries DataFrame
groceries_df = pd.read_csv('../Resources/groceries.csv')
ev_df = pd.read_csv('../Resources/evcs.csv')
bars_df = pd.read_csv('../Resources/nyc_bars.csv')  

In [76]:

ev_df['Type'] = 'EVCS'
clean_ev_df = ev_df[['City', 'Station Name', 'Street Address', 'Type']]
clean_ev_df.head()

,City,Station Name,Street Address,Type
0,Fair Haven,Village of Fairhaven,14523 Cayuga St,EVCS
1,Hudson,The Wick Hotel - Tesla Destination,41 Cross St,EVCS
2,Tupper Lake,Faust Motel - Tesla Destination,151 Main St,EVCS
3,Delmar,LIVEWELL GROUP 708 CATLYN CT,708 Catlyn Ct,EVCS
4,Midland Beach,NYC FLEET DPRMBLOT8_1_L3,920 Father Capodanno Blvd,EVCS


In [77]:
bars_df['Type'] = 'Bar'
bars_df.columns

Index(['City', 'Name', 'Address', 'Type'], dtype='object')

In [78]:
groceries_df["Type"]= "Grocery"
groceries_df.columns

Index(['City', 'Name', 'Address', 'Type'], dtype='object')

In [79]:

# Group by 'Borough' and count the types of groceries
boro_counts = groceries_df.groupby('City')['Type'].value_counts().reset_index(name='count')

# Sort the DataFrame based on 'count' in descending order
boro_counts_sorted = boro_counts.sort_values(by=['City', 'count'], ascending=[True, False])

# Print the sorted DataFrame
print(boro_counts_sorted)


            City     Type  count
0          Bronx  Grocery   1000
1       Brooklyn  Grocery   1000
2      Manhattan  Grocery   1000
3         Queens  Grocery   1000
4  Staten Island  Grocery   1000


In [80]:

# Sample data for demonstration
data = {
    'City': ['Manhattan', 'Queens', 'Brooklyn', 'Bronx', 'Staten Island', ],
    'Type': ['EVCS', 'EVCS', 'EVCS', 'EVCS', 'EVCS', ],
}

ev_df = pd.DataFrame(data)

# List of cities you want to include
selected_cities = ['Manhattan', 'Queens', 'Brooklyn', 'Bronx']

# Filter the DataFrame to include only the selected cities
filtered_df = ev_df[ev_df['City'].isin(selected_cities)]

# Group by 'City' and count the types of groceries
boro_counts = filtered_df.groupby('City')['Type'].value_counts().reset_index(name='count')

# Sort the DataFrame based on 'City' and 'count' in descending order
boro_counts_sorted = boro_counts.sort_values(by=['City', 'count'], ascending=[True, False])

# Print the sorted DataFrame
print(boro_counts_sorted)


        City  Type  count
0      Bronx  EVCS      1
1   Brooklyn  EVCS      1
2  Manhattan  EVCS      1
3     Queens  EVCS      1
